In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms 
from statsmodels.iolib.summary2 import summary_col

c:\python\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


## Ejemplo 13.1 La fertilidad de las mujeres en el tiempo (fertil1.dta) 

In [5]:
dfej13_1 = pd.read_stata('fertil1.dta')
dfej13_1.head()

,year,educ,meduc,feduc,age,kids,black,east,northcen,west,...,y80,y82,y84,agesq,y74educ,y76educ,y78educ,y80educ,y82educ,y84educ
0,72,12,8,8,48,4,0,0,1,0,...,0,0,0,2304,0,0,0,0,0,0
1,72,17,8,18,46,3,0,0,0,0,...,0,0,0,2116,0,0,0,0,0,0
2,72,12,7,8,53,2,0,0,1,0,...,0,0,0,2809,0,0,0,0,0,0
3,72,12,12,10,42,2,0,0,1,0,...,0,0,0,1764,0,0,0,0,0,0
4,72,12,3,8,51,2,0,0,0,0,...,0,0,0,2601,0,0,0,0,0,0


In [6]:
dfej13_1.dtypes

year         int8
educ         int8
meduc        int8
feduc        int8
age          int8
kids         int8
black        int8
east         int8
northcen     int8
west         int8
farm         int8
othrural     int8
town         int8
smcity       int8
y74          int8
y76          int8
y78          int8
y80          int8
y82          int8
y84          int8
agesq       int16
y74educ      int8
y76educ      int8
y78educ      int8
y80educ      int8
y82educ      int8
y84educ      int8
dtype: object

In [7]:
dfej13_1['kids'] = dfej13_1['kids'].astype(float)

In [8]:
dfej13_1['educ'] = dfej13_1['educ'].astype(float)

In [9]:
dfej13_1['age'] = dfej13_1['age'].astype(float)

In [10]:
dfej13_1['black'] = dfej13_1['black'].astype(float)

In [11]:
dfej13_1['east'] = dfej13_1['east'].astype(float)

In [12]:
dfej13_1['northcen'] = dfej13_1['northcen'].astype(float)

In [13]:
dfej13_1['farm'] = dfej13_1['farm'].astype(float)

In [14]:
dfej13_1['othrural'] = dfej13_1['othrural'].astype(float)

In [15]:
dfej13_1['town'] = dfej13_1['town'].astype(float)

In [16]:
dfej13_1['smcity'] = dfej13_1['smcity'].astype(float)

In [17]:
regdfej13_1 = smf.ols('kids ~  educ + age + np.square(age) + black + east + northcen+ west + farm +othrural + town + smcity + y74 + y76 + y78 +y80 + y82+ y84 ', data = dfej13_1).fit()
print(regdfej13_1.summary())

                            OLS Regression Results                            
Dep. Variable:                   kids   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     9.723
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           2.42e-24
Time:                        12:06:59   Log-Likelihood:                -2091.2
No. Observations:                1129   AIC:                             4218.
Df Residuals:                    1111   BIC:                             4309.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -7.7425      3.052     -2.

In [ ]:
# Comentario : Se verifica el patrón de los coeficientes de las dummies temporales para ver la evoución del promedio de 
# fertilidad (dependiente) , controlando por todas las otras independientes
# Los coeficientes de las dummies muestran descenso de la fertilidad marcado desde los 80

In [19]:
# dummy 1982
regdfej13_1.params[16]

-0.5224841991276494

In [ ]:
# ceteris paribus una mujer tuvo en promedio, 0.52 hijos menos en 1982 que en 1972, año base. Es decir, de 100 mujeres
# habría 52 hijos menos
# Coo está controlada la educación y las otras variables, ese descenso es por otras razones, no capturadas en las x de control

In [ ]:
# Las binarias de los años son soignificativas conjuntamente

In [14]:
hypotheses = '(y74 =y76 = y78 = y80 = y82 = y84 = 0)'
f_test = regdfej13_1.f_test(hypotheses)
print(f_test)

<F test: F=array([[5.86950867]]), p=4.855189867572506e-06, df_denom=1111, df_num=6>


In [ ]:
# La variable educación indica que con más educación se tiene menos hijos

In [21]:
regdfej13_1.params[1]

-0.12842683104662517

In [ ]:
# Si comparamos una mujer universitaria con una secundaria, es decir con 4 años más de educción, el efecto es 

In [22]:
regdfej13_1.params[1]*4

-0.5137073241865007

In [ ]:
# es deci 100 universitarias tendrán 51 menos hijos que las secundarias

In [ ]:
# La vairable edad tiene una parábola cuyo máximo es 46 años, donde la mayoría ha dejado de tener hijos

In [ ]:
# dKids/dedad = b(edad)- 2*b(edad^2)

In [24]:
# regdfej13_1.params[2]+2*regdfej13_1.params[3]*(edad) = 0 
edadmax = -regdfej13_1.params[2]/(2*regdfej13_1.params[3])
edadmax

45.842030306112406

In [ ]:
# Heterocedasrticidad

In [15]:
bptest = sms.diagnostic.het_breuschpagan(regdfej13_1.resid, regdfej13_1.model.exog)
df2 = pd.DataFrame({'Chi-Sq':[bptest[0]],
                   'Prob>Chi-Sq':[bptest[1]]})
print(df2)

      Chi-Sq  Prob>Chi-Sq
0  55.315373     0.000006


In [ ]:
# FALTA. reVISAR CON CAP 8 

## Ejemplo 13.2 Cambios en los efectos de la educación y de la diferencia salarial por razón de sexo (cps78_85.dta) 

In [30]:
dfej13_2 = pd.read_stata('cps78_85.dta')
dfej13_2.head()

,educ,south,nonwhite,female,married,exper,expersq,union,lwage,age,year,y85,y85fem,y85educ,y85union
0,12,0,0,0,0,8,64,0,1.2150,25,78,0,0,0,0
1,12,0,0,1,1,30,900,1,1.6094,47,78,0,0,0,0
2,6,0,0,0,1,38,1444,1,2.1401,49,78,0,0,0,0
3,12,0,0,0,1,19,361,1,2.0732,36,78,0,0,0,0
4,12,0,0,0,1,11,121,0,1.6490,28,78,0,0,0,0


In [31]:
dfej13_2.dtypes

educ           int8
south          int8
nonwhite       int8
female         int8
married        int8
exper          int8
expersq       int16
union          int8
lwage       float32
age            int8
year           int8
y85            int8
y85fem         int8
y85educ        int8
y85union       int8
dtype: object

In [32]:
dfej13_2['educ'] = dfej13_2['educ'].astype(float)

In [33]:
dfej13_2['exper'] = dfej13_2['exper'].astype(float)

In [34]:
dfej13_2['union'] = dfej13_2['union'].astype(float)

In [35]:
dfej13_2['female'] = dfej13_2['female'].astype(float)

In [ ]:
dfej13_2['y85educ´] = dfej13_2['y85']*dfej13_2['educ']
dfej13_2['y85female'] = dfej13_2['y85']*dfej13_2['female']

In [36]:
# Creo serie para no agregar en la base
y85educ = dfej13_2['y85']*dfej13_2['educ']
y85female = dfej13_2['y85']*dfej13_2['female']

In [37]:
regdfej13_2 = smf.ols('lwage ~  y85 + educ +y85educ + exper +np.square(exper) + union + female + y85female ', data = dfej13_2).fit()
print(regdfej13_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     99.80
Date:                Sat, 10 Oct 2020   Prob (F-statistic):          4.46e-124
Time:                        14:56:17   Log-Likelihood:                -574.24
No. Observations:                1084   AIC:                             1166.
Df Residuals:                    1075   BIC:                             1211.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.4589      0.093  

In [ ]:
# Comentario : Se usan variables dummy de interacción para saber si las independientes cambian su efecto en el tiempo
# En el ejemplo se ve si la rentabilidad en la educación y las diferencias de género cambian en el tiempo
# El año base es 78 . Hay dummy de año, y dos dummies de interacción , con educ, y female
# Se ha supuesto que la sindicalización y la experiencia tienen el mismo efecto en los salarios en ambos períodos

In [ ]:
# Ordenada para 78

In [38]:
regdfej13_2.params[0]

0.45893288315285313

In [ ]:
# ORdenada para 85

In [39]:
regdfej13_2.params[0]+regdfej13_2.params[1]

0.5767391010172603

In [ ]:
# Impacto de la educación en 78, alrededor de 7.5%

In [40]:
regdfej13_2.params[2]

0.07472091292293376

In [ ]:
# Impacto de la educación en 85, es de 9.3%

In [41]:
regdfej13_2.params[2]+regdfej13_2.params[3]

0.09318144522805318

In [ ]:
# p value de interacción y85*edu 

In [45]:
regdfej13_2.pvalues[3]

0.0486934422862361

In [ ]:
# la diferencia en el impacto de la educación es estadísticamente significativa al nivel de 5% contra 
# una alternativa de dos colas

In [ ]:
# Diferencia en log(wage) en 78. Ceteris paribus una mujer gana 31.7% menos (en log(wage))

In [42]:
regdfej13_2.params[7]

-0.31670864807359267

In [ ]:
# Falta transformar a wage

In [43]:
# Diferencia en log(wage) en 85. La mujer gana 23.2% menos (en log(wage))

In [44]:
regdfej13_2.params[7]+regdfej13_2.params[8]

-0.23165667751274965

In [ ]:
# es decir, la diferencia de género parace haber bajado 8.5% 

In [ ]:
# p value de interacción y85*female 

In [46]:
regdfej13_2.pvalues[8]

0.09768120660347125

In [ ]:
# es significativo al nivel de 5% contra una alternativa de una cola positiva.

In [ ]:
# Falta explicar porque cuando la dependiente está en logaritmo y es en valor monetario , no hace falta poner en real

In [ ]:
# Falta ver todas las interacciones igual a ecuaciones separadas

In [ ]:
# Análisis de políticas con dos cortes trasversales

## Ejemplo 13.3 Efectos de la ubicación de un incinerador de basura en el precio de la vivienda (kielmc.dta)

In [47]:
dfej13_3 = pd.read_stata('kielmc.dta')
dfej13_3.head()

,year,age,agesq,nbh,cbd,intst,lintst,price,rooms,area,...,lprice,y81,larea,lland,y81ldist,lintstsq,nearinc,y81nrinc,rprice,lrprice
0,1978,48,2304.0,4,3000.0,1000.0,6.9078,60000.0,7,1660,...,11.002100,0,7.414573,8.429017,0.0,47.717705,1,0,60000.0,11.002100
1,1978,83,6889.0,4,4000.0,1000.0,6.9078,40000.0,6,2612,...,10.596635,0,7.867871,9.032409,0.0,47.717705,1,0,40000.0,10.596635
2,1978,58,3364.0,4,4000.0,1000.0,6.9078,34000.0,6,1144,...,10.434115,0,7.042286,8.517193,0.0,47.717705,1,0,34000.0,10.434115
3,1978,11,121.0,4,4000.0,1000.0,6.9078,63900.0,5,1136,...,11.065075,0,7.035269,9.210340,0.0,47.717705,1,0,63900.0,11.065075
4,1978,48,2304.0,4,4000.0,2000.0,7.6009,44000.0,5,1868,...,10.691945,0,7.532624,9.210340,0.0,57.773682,1,0,44000.0,10.691945


In [50]:
dfej13_3.dtypes

year          int16
age           int16
agesq       float32
nbh            int8
cbd         float32
intst       float32
lintst      float32
price       float32
rooms          int8
area          int16
land        float32
baths          int8
dist        float32
ldist       float32
wind           int8
lprice      float32
y81            int8
larea       float32
lland       float32
y81ldist    float32
lintstsq    float32
nearinc     float64
y81nrinc       int8
rprice      float32
lrprice     float32
dtype: object

In [48]:
dfej13_3['nearinc'] = dfej13_3['nearinc'].astype(float)

In [49]:
# Se filtra la base para el año 1981
dfej13_381 = dfej13_3[dfej13_3.year==1981]

In [50]:
regdfej13_3 = smf.ols('rprice ~  nearinc ', data = dfej13_381).fit()
print(regdfej13_3.summary())

                            OLS Regression Results                            
Dep. Variable:                 rprice   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     27.73
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           5.14e-07
Time:                        16:03:51   Log-Likelihood:                -1670.1
No. Observations:                 142   AIC:                             3344.
Df Residuals:                     140   BIC:                             3350.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.013e+05   3093.027     32.754      0.0

In [ ]:
# Comentario : Este modelo (inocente) es del valor de las casas contra una dummy de cera vs lejos, por lo que la ordenada es 
# el valor promedio de las que están lejos y el coeficiente de la dummy es la diferencia de las que están cerca
# el p valu muestra que hay disferencia significativa entre los dos grupos
# Las cercanas valen 30000 menos

In [ ]:
# Problema: esa ecuación no indica que el incinerador sea el que provoque la diferencia
# Si hacemos la regresión en el 78, veremos que igual había diferencia y era significativa

In [54]:
# Se filtra la base para el año 1978
dfej13_378 = dfej13_3[dfej13_3.year==1978]

In [55]:
regdfej13_3a = smf.ols('rprice ~  nearinc ', data = dfej13_378).fit()
print(regdfej13_3a.summary())

                            OLS Regression Results                            
Dep. Variable:                 rprice   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     15.74
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           0.000105
Time:                        16:29:36   Log-Likelihood:                -2094.9
No. Observations:                 179   AIC:                             4194.
Df Residuals:                     177   BIC:                             4200.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.252e+04   2653.790     31.094      0.0

In [ ]:
# Comentario : Nuevamente la diferencia es sginficiativa, y en este caso es de 18000 menos para las cercanas
# Coherente con que se construyó el incinerador en zona de casas de bajo valor

In [53]:
print(summary_col([regdfej13_3, regdfej13_3a],stars=True,float_format='%0.3f',
                  model_names=['grab81\n(b/se)','grab78\n(b/se)'],
                 info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared), 
                           'Adj.R2':lambda x: "{:.3f}".format(x.rsquared_adj)}))


              grab81        grab78   
              (b/se)        (b/se)   
-------------------------------------
Intercept 101307.514*** 82517.228*** 
          (3093.027)    (2653.790)   
nearinc   -30688.274*** -18824.370***
          (5827.709)    (4744.594)   
N         142           179          
R2        0.165         0.082        
Adj.R2    0.159         0.076        
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [ ]:
# Cómo saber si el incinerador reduce el valor de las casas? 

In [ ]:
# La clave es ver el cambio en el coeficiente de la dummy

In [57]:
Difendif = regdfej13_3.params[1]-regdfej13_3a.params[1]
Difendif

-11863.903252112581

$ \hat \delta_{1} = (\overline{rprice_{n81}}- \overline{rprice_{f81}})-( \overline{rprice_{n78}} - \overline{rprice_{f78}}) $

In [ ]:
# Para encontrar el error estándar de ese coeficiente es mejor hacer la regresión con las Dos Secciones Cruzadas y usando una 
# dummy de tiempo, cuya interacción con la dummy de distancia es ese coeficiente

In [59]:
# Creo serie para no agregar en la base
y81nearinc = dfej13_3['y81']*dfej13_3['nearinc']

In [60]:
regdfej13_3b = smf.ols('rprice ~  y81 + nearinc + y81nearinc ', data = dfej13_3).fit()
print(regdfej13_3b.summary())

                            OLS Regression Results                            
Dep. Variable:                 rprice   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     22.25
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           4.22e-13
Time:                        16:52:53   Log-Likelihood:                -3765.2
No. Observations:                 321   AIC:                             7538.
Df Residuals:                     317   BIC:                             7554.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.252e+04   2726.910     30.260      0.0

In [ ]:
# Comentario : Vemos que el coeficiente de la interacción de las dos dummies es lo esperado, la DifenDif y que no es 
# estadísticamente muy significativo
# El coeficiente de la dummy de distancia muestra los otros efectos que hacen a la diferencia de valores pro distancia, 
# distintos al incinerador
# La ordenada, mide el promedio de valor de las casas lejanas en el 78

In [ ]:
# El coeficiente de la interacción de las dos dummies mide la disminución en valor por el inicinerador si ambos grupos de casas
# no se revalorizaron pro otras razones

In [ ]:
# Otras características

In [ ]:
# Es probable que el grupo de casas del 81 sean sistemáticamente distintas a las del 78, por lo que interesa controlar por 
# esas características que pueden valorarlas diferente
# Aún si no difieren, es bueno incluir esas características para reducir la varianza del error y quizás haga más significativo
# al coeficiente de DifenDif

In [61]:
regdfej13_3c = smf.ols('rprice ~  y81 + nearinc + y81nearinc + age + agesq ', data = dfej13_3).fit()
print(regdfej13_3c.summary())

                            OLS Regression Results                            
Dep. Variable:                 rprice   R-squared:                       0.414
Model:                            OLS   Adj. R-squared:                  0.405
Method:                 Least Squares   F-statistic:                     44.59
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           1.00e-34
Time:                        17:15:20   Log-Likelihood:                -3710.0
No. Observations:                 321   AIC:                             7432.
Df Residuals:                     315   BIC:                             7455.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.912e+04   2406.051     37.039      0.0

In [ ]:
# Comentario : se ha controlado la edad de las casas y aumenta la R2 (al reducir la varianza residual). 
# El coeficiente de y81·nearinc ahora es mayor y su error estándar es menor.

In [63]:
regdfej13_3d = smf.ols('rprice ~  y81 + nearinc + y81nearinc + age + agesq + intst+ land+ area+ rooms+ baths ', data = dfej13_3).fit()
print(regdfej13_3d.summary())

                            OLS Regression Results                            
Dep. Variable:                 rprice   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     60.19
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           1.16e-66
Time:                        17:20:20   Log-Likelihood:                -3622.7
No. Observations:                 321   AIC:                             7267.
Df Residuals:                     310   BIC:                             7309.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.381e+04   1.12e+04      1.237      0.2

In [ ]:
# Comentario : ahora se controla también por distancia a la carretera (intst), el área del terreno (land), 
# el área de la casa (area), el número de habitaciones (rooms) y el número de baños (baths).
# La estimación de la DifenDif es más cercana a cuando se computó sin controles, pero con error estándar menor y que la hace 
# ahora si , significtiva

In [ ]:
# Todos los errores estándar son menores, como se esperaba, salvo para la ordenada , pero en realidad esta no importa si 
# están todos los controles necesarios
# Que la dummy de distancia sola no sea significativa es porque los controles capturan las principales características 
# que explican la diferencia de precios

In [ ]:
# Para medir los efectos en términos porcentuales mejor es con el log del precio

In [ ]:
# Cuando se hace en logaritmo , ahora el coeficiente de DifenDif es multiplicado por 100 y se interpreta como diferencia
# porcentual

In [65]:
regdfej13_3e = smf.ols('np.log(rprice) ~  y81 + nearinc + y81nearinc ', data = dfej13_3).fit()
print(regdfej13_3e.summary())

                            OLS Regression Results                            
Dep. Variable:         np.log(rprice)   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     34.47
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           2.62e-19
Time:                        17:35:41   Log-Likelihood:                -105.68
No. Observations:                 321   AIC:                             219.4
Df Residuals:                     317   BIC:                             234.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.2854      0.031    369.839      0.0

In [ ]:
# Comentario : Las casas cercanas al incinerador , pierden por él, 6.3% de su valor. Aunque la significatividad es mala, 
# vimos que cuando se usan todos los controles cambia 

In [66]:
regdfej13_3f = smf.ols('np.log(price) ~  y81 + nearinc + y81nearinc ', data = dfej13_3).fit()
print(regdfej13_3f.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(price)   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                     73.15
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           5.61e-36
Time:                        17:37:20   Log-Likelihood:                -105.68
No. Observations:                 321   AIC:                             219.4
Df Residuals:                     317   BIC:                             234.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.2854      0.031    369.839      0.0

In [ ]:
# Comentario: Com ovimos, cuando usamos log con precios nominales, solamente difiere del uso en terminos reales en el 
# coeficiente de la dummy temporal

In [67]:
regdfej13_3g = smf.ols('np.log(price) ~  y81 + nearinc + y81nearinc + age + agesq + np.log(intst)+ np.log(land)+ np.log(area)+ rooms+ baths', data = dfej13_3).fit()
print(regdfej13_3g.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(price)   R-squared:                       0.790
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                     116.9
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           6.53e-99
Time:                        17:50:50   Log-Likelihood:                 60.690
No. Observations:                 321   AIC:                            -99.38
Df Residuals:                     310   BIC:                            -57.89
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.6517      0.416     18.399

In [ ]:
# Comentario : Ahora vemos que el efecto del incinerador es de 13% al reducir el valor de las propiedades cercanas
# y significativo al colocar los controles como antes, pues eso reduce la varianza del error

## Ejemplo 13.4  Efecto de las leyes de indemnización de los trabajadores sobre la duración  de la baja laboral (injury.dta)

In [ ]:
# Una indemnización mayor para los de mayor ingreso hace que permanezcan sin trabajar por más tiempo? 

In [68]:
dfej13_4 = pd.read_stata('injury.dta')
dfej13_4.head()

,durat,afchnge,highearn,male,married,hosp,indust,injtype,age,prewage,...,head,neck,upextr,trunk,lowback,lowextr,occdis,manuf,construc,highlpre
0,1.0,1,1,1.0,0.0,1,3.0,1,26.0,404.950012,...,1,0,0,0,0,0,0,0.0,0.0,6.003764
1,1.0,1,1,1.0,1.0,0,3.0,1,31.0,643.825012,...,1,0,0,0,0,0,0,0.0,0.0,6.467427
2,84.0,1,1,1.0,1.0,1,3.0,1,37.0,398.125000,...,1,0,0,0,0,0,0,0.0,0.0,5.986766
3,4.0,1,1,1.0,1.0,1,3.0,1,31.0,527.799988,...,1,0,0,0,0,0,0,0.0,0.0,6.268717
4,1.0,1,1,1.0,1.0,0,3.0,1,23.0,528.937500,...,1,0,0,0,0,0,0,0.0,0.0,6.270870


In [69]:
afhi = dfej13_4['afchnge']*dfej13_4['highearn']

In [70]:
regdfej13_4 = smf.ols('np.log(durat) ~  afchnge + highearn + afhi ', data = dfej13_4).fit()
print(regdfej13_4.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(durat)   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     38.34
Date:                Sat, 10 Oct 2020   Prob (F-statistic):           1.42e-24
Time:                        19:51:56   Log-Likelihood:                -12011.
No. Observations:                7150   AIC:                         2.403e+04
Df Residuals:                    7146   BIC:                         2.406e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.1993      0.027     44.241      0.0

In [ ]:
# Comentario : El grupo de control, son los de bajo ingreso, siendo la dummy higheard. 
# La variable temporal, es afchange, y la variable  de interacción es afhi
# El coeficiente de la afhi, es decir la difendif indica que la duración promedio aumento caso 19% con el aumento del seguro
# El coeficiente de afchange es chico y no significativo, esperable pues no hay cambio en el tiempo para los de bao ingreso
# El bajo R2 se explica pues el cambio en la duración se explica por muchos factores, como la gravedad de la lesión, género,
# estado civil, industria, que oermite considerar que  las personas y las lesiones difieren sistemáticamente en los dos años
# Como la muestra es grande, el t es significativo

In [ ]:
# Panel en dos años
# Método Diferencias

## Ejemplo 13.5  ¿Dormir o trabajar?  (slp75_81.dta)

$slpnap_{it} = \beta_{0}+ \delta_{0} d281_{t}+ \beta_{1} totwrk_{it}+ \beta_{2} educa_{it}+ \beta_{3}marr_{it}+\beta_{4}ymgkid_{it}+ \beta_{5}gdhlth_{it}+ a_{i} + u_{it} $  t = 1,2

In [ ]:
# Importante dejar correlacionar ai con totwrkit pues fatores por ejemplo biológicos que hacen dormir, captados en ai, 
# se correlacionen con el tiempo de trabajo
# No se incluye sexo o raza , que si iría en sección cruzada, pues no cambia en el tiempo, y están dentro de ai

In [ ]:
# La diferenciación 

$\Delta slpnap_{i} = \Delta_{0} + \beta_{1} \Delta totwrk_{i}+ \beta_{2} \Delta educ_{i}+ \beta_{3} \Delta marr_{i}+\beta_{4} \Delta ymgkid_{i}+ \beta_{5} \Delta gdhlth_{i}+ \Delta u_{i} $ 

In [ ]:
# supuesto qeu la variación del error no se correlaciona con las variaciones en las explicativas

In [71]:
dfej13_5 = pd.read_stata('slp75_81.dta')
dfej13_5.head()

,age75,educ75,educ81,gdhlth75,gdhlth81,male,marr75,marr81,slpnap75,slpnap81,totwrk75,totwrk81,yngkid75,yngkid81,ceduc,cgdhlth,cmarr,cslpnap,ctotwrk,cyngkid
0,46,16,16,1,1,1,0,1,3991,4695,2050,2430,0,0,0,0,1,704,380,0
1,39,16,16,1,1,1,1,0,2243,2195,2713,2610,0,0,0,0,-1,-48,-103,0
2,55,15,15,1,0,1,1,1,3285,3115,2493,0,0,0,0,-1,0,-170,-2493,0
3,39,16,16,1,1,1,1,1,3158,3387,2778,1787,0,0,0,0,0,229,-991,0
4,54,17,17,1,1,1,1,1,3743,3800,3118,552,0,0,0,0,0,57,-2566,0


In [72]:
dslpnap = dfej13_5['slpnap75']-dfej13_5['slpnap81']

In [73]:
dtotwrk = dfej13_5['totwrk75']-dfej13_5['totwrk81']

In [74]:
deduc = dfej13_5['educ75']-dfej13_5['educ81']

In [75]:
dmarr = dfej13_5['marr75']-dfej13_5['marr81']

In [76]:
dyngkid = dfej13_5['yngkid75']-dfej13_5['yngkid81']

In [77]:
dgdhlth = dfej13_5['gdhlth75']-dfej13_5['gdhlth81']

In [ ]:
# La base de datos ya estaba preparada y facilitó crear las variables diferencia

In [78]:
regdfej13_5 = smf.ols('dslpnap ~  dtotwrk + deduc+ dmarr+ dyngkid+ dgdhlth ', data = dfej13_5).fit()
print(regdfej13_5.summary())

                            OLS Regression Results                            
Dep. Variable:                dslpnap   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     8.191
Date:                Sun, 11 Oct 2020   Prob (F-statistic):           3.83e-07
Time:                        12:44:56   Log-Likelihood:                -1864.4
No. Observations:                 239   AIC:                             3741.
Df Residuals:                     233   BIC:                             3762.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     92.6340     45.866      2.020      0.0

In [ ]:
# Comentarios: El coeficiente de la variación de totwrk indica que una hora más de trabajo se relaciona con 0.23 horas 
# menos de sueño , siendo significativo
# Salvo la ordenada, el resto no es significativo

In [79]:
hypotheses = '(deduc= dmarr= dyngkid= dgdhlth  = 0)'
f_test = regdfej13_5.f_test(hypotheses)
print(f_test)

<F test: F=array([[0.86483857]]), p=0.48573468205905057, df_denom=233, df_num=4>


In [ ]:
# El test de significatividad conjunta de todas menos totwrk, (que no son significativas individualmente) muestra que 
# no son conjuntamente significativas y que podrían omitirse de la ecuación

In [ ]:
# La variación de educación tiene un error estandar relativamente muy grande y es porque no tiene mucha varaición

In [82]:
deduc.describe()

count    239.000000
mean      -0.372385
std        0.804028
min       -5.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        0.000000
dtype: float64

In [85]:
deduc. value_counts(normalize=True)

 0    0.765690
-1    0.133891
-2    0.083682
-5    0.008368
-3    0.004184
-4    0.004184
dtype: float64

In [ ]:
# El 76% no cambió si nivel educativo y el 90% solo a lo sumo un año

In [ ]:
# El gran error estándar muestra que no hay suficiente variación para estimar el coeficiente con precisión
# De todos modos el coeficiente es muy pequeño

In [ ]:
# Panel para rezagos distribuidos 

In [ ]:
# Ejemplo de dos años y método de diferencias

## Ejemplo 13.6  Modelo de retardos dlstrlbuldos del índice de dellncuencla en el porcentaje de casos resueltos (crime3.dta)

In [2]:
dfej13_6 = pd.read_stata('crime3.dta')
dfej13_6.head()

,district,year,crime,clrprc1,clrprc2,d78,avgclr,lcrime,clcrime,cavgclr,cclrprc1,cclrprc2
0,1,72,49.540001,22,23,0,22.5,3.902781,NaN,NaN,NaN,NaN
1,1,78,71.320000,15,17,1,16.0,4.267177,0.364396,-6.5,-7.0,-6.0
2,2,72,14.770000,51,62,0,56.5,2.692598,NaN,NaN,NaN,NaN
3,2,78,17.850000,39,40,1,39.5,2.882004,0.189405,-17.0,-12.0,-22.0
4,3,72,17.350000,33,34,0,33.5,2.853593,NaN,NaN,NaN,NaN


In [ ]:
# Tasa de delincuencia para años 72 y 78 en función de los rezagos del porcentaje de casos resueltos
# En la base ya están las primeras diferencias de los casos resueltos
# Como la base es de un panel de distritos policíacos, tenemos efectos fijos

$ log(crime)_{it} = \beta_{0}+ \delta_{0} d78_{t} + \beta_{1} clrpcr_{i,t-1}+ \beta_{2} clrpcr_{i,t-2}+a_{i}+ u_{it} $

In [ ]:
# Diferenciando

$ \Delta log(crime)_{i} = \delta_{0} + \beta_{1} \Delta clrpcr_{-1}+ \beta_{2} \Delta clrpcr_{-2}+ \Delta u_{i} $

In [3]:
regdfej13_6 = smf.ols('clcrime ~  cclrprc1 + cclrprc2  ', data = dfej13_6).fit()
print(regdfej13_6.summary())

                            OLS Regression Results                            
Dep. Variable:                clcrime   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     5.992
Date:                Thu, 22 Oct 2020   Prob (F-statistic):            0.00465
Time:                        23:07:37   Log-Likelihood:                -17.194
No. Observations:                  53   AIC:                             40.39
Df Residuals:                      50   BIC:                             46.30
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0857      0.064      1.343      0.1

In [ ]:
# Comentario : los rezagos sonnegativos indicando que afectan a la tasa de crimen y es el segundo que es significativo. 
# Si aumentan los casos resueltos en 10 puntos hace dos años, 13,2% es lo que disminuyen la tasa de delincuencia

In [7]:
dfej13_6.clcrime

0           NaN
1      0.364396
2           NaN
3      0.189405
4           NaN
         ...   
101   -0.393374
102         NaN
103   -0.079920
104         NaN
105    0.497440
Name: clcrime, Length: 106, dtype: float32

In [6]:
np.log(dfej13_6.crime).diff()

0           NaN
1      0.364396
2     -1.574579
3      0.189405
4     -0.028411
         ...   
101   -0.393374
102   -0.442677
103   -0.079920
104    0.826978
105    0.497440
Name: crime, Length: 106, dtype: float32

In [14]:
dfej13_6.groupby('district')['lcrime'].diff()

0           NaN
1      0.364396
2           NaN
3      0.189405
4           NaN
         ...   
101   -0.393374
102         NaN
103   -0.079920
104         NaN
105    0.497440
Name: lcrime, Length: 106, dtype: float32

In [ ]:
# Otra forma de hacer la regresión con diff(), por ejemplo con el primer rezago
# Pero vemos que es porque dejamos cclrprc2 y ello hace qeu sus nan elimine de la regresión los nan que con diff() no pudimos
# eliminar

In [4]:
regdfej13_6a = smf.ols('np.log(crime).diff() ~  clrprc1.diff() + cclrprc2  ', data = dfej13_6).fit()
print(regdfej13_6a.summary())

                             OLS Regression Results                             
Dep. Variable:     np.log(crime).diff()   R-squared:                       0.193
Model:                              OLS   Adj. R-squared:                  0.161
Method:                   Least Squares   F-statistic:                     5.992
Date:                  Thu, 22 Oct 2020   Prob (F-statistic):            0.00465
Time:                          23:07:44   Log-Likelihood:                -17.194
No. Observations:                    53   AIC:                             40.39
Df Residuals:                        50   BIC:                             46.30
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.085

In [ ]:
# Ahora mostramos claramente que no está bien la regresión pues ahora no se han eliminado las diferencias que no corresponden 
# pues se hacen con diferentes individuos

In [8]:
regdfej13_6b = smf.ols('np.log(crime).diff() ~  clrprc1.diff() + clrprc2.diff()  ', data = dfej13_6).fit()
print(regdfej13_6b.summary())

                             OLS Regression Results                             
Dep. Variable:     np.log(crime).diff()   R-squared:                       0.440
Model:                              OLS   Adj. R-squared:                  0.429
Method:                   Least Squares   F-statistic:                     40.11
Date:                  Thu, 22 Oct 2020   Prob (F-statistic):           1.40e-13
Time:                          23:12:50   Log-Likelihood:                -72.336
No. Observations:                   105   AIC:                             150.7
Df Residuals:                       102   BIC:                             158.6
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.002

In [ ]:
# Evaluación de politicas con panel de dos períodos

## Ejemplo 13.7 Efecto sobre los accidentes mortales de tráfico de las leyes sobre la conducción en estado de embriaguez (traffic1.dta)

In [88]:
dfej13_7 = pd.read_stata('traffic1.dta')
dfej13_7.head()

,state,admn90,admn85,open90,open85,dthrte90,dthrte85,speed90,speed85,cdthrte,cadmn,copen,cspeed
0,AL,0,0,0,0,2.6,2.9,1,0,-0.3,0,0,1
1,AK,1,1,1,0,2.1,3.2,0,0,-1.1,0,1,0
2,AZ,1,0,0,0,2.5,4.4,1,0,-1.9,1,0,1
3,AR,0,0,0,0,2.9,3.4,1,0,-0.5,0,0,1
4,CA,1,0,1,1,2.0,2.6,1,0,-0.6,1,0,1


In [ ]:
# Se busca explicar las muertes en accidentes de tránsito en función de la ley de recipientes abiertos y leyes administrativas
# Es un panel de estados
# Son datos de 1985 y 1990
# Las variables ya están separadas por años

In [ ]:
# Si hiciéramos análisis con un solo corte trasversal , entre muertes y las binarias que indiquen la presencia de las leyes
# es probable que no funciones pues se decide las leyes por accidentes de últimos años, por lo que estaría correlacionado con 
# la causalidad del otro lado y no estaríamos estudiando el efecto de las leyes sobre accidentes

In [ ]:
# Mejor es estudair con un panel, en un período donde algunos estados adoptan las leyes y otros quizás revoquen las que tienen

In [93]:
dfej13_7.loc[dfej13_7.open85==1,'open85' ].sum()

19

In [95]:
dfej13_7.loc[dfej13_7.open90==1,'open90' ].sum()

22

In [96]:
dfej13_7.loc[dfej13_7.admn85==1,'admn85' ].sum()

21

In [97]:
dfej13_7.loc[dfej13_7.admn90==1,'admn90' ].sum()

29

In [89]:
ddthrte = dfej13_7['dthrte90']-dfej13_7['dthrte85'] 

In [90]:
dopen = dfej13_7['open90']-dfej13_7['open85'] 

In [91]:
dadmn = dfej13_7['admn90']-dfej13_7['admn85'] 

In [92]:
regdfej13_7 = smf.ols('ddthrte ~  dopen + dadmn  ', data = dfej13_7).fit()
print(regdfej13_7.summary())

                            OLS Regression Results                            
Dep. Variable:                ddthrte   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     3.231
Date:                Sun, 11 Oct 2020   Prob (F-statistic):             0.0482
Time:                        19:09:46   Log-Likelihood:                -16.323
No. Observations:                  51   AIC:                             38.65
Df Residuals:                      48   BIC:                             44.44
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4968      0.052     -9.476      0.0

In [ ]:
# Comentario : La estimación con diferencias sugiere que las leyes de recipiente abiertos disminuyeron los accidentes en 
# 0.42 , significativa al 5% 
# Las leyes admnistrativas tienen también disminuyen los accidentes, pero menos significatividad
# LA ordenada muestra qeu las muertes por accidente bajan considerablemente para todos en ese período, se hayan o no puesto leyes
# Los que pusieron leyes de recipientes abiertos disminuyeron más
# Hay otras leyes y otros controles que podrían ponerse y que afectan la tasa de accidentes, y podrían ponerse

In [98]:
dfej13_7.dthrte85.mean()

2.7000003

In [99]:
dfej13_7.dthrte85.std()

0.6033241

In [ ]:
# Evaluación de politicas con panel de más de dos períodos

## Ejemplo 13.8 Efectos de los polígonos industriales en las solicitudes del seguro de desempleo (ezunem.dta)

In [100]:
dfej13_8 = pd.read_stata('ezunem.dta')
dfej13_8.head()

,year,uclms,ez,d81,d82,d83,d84,d85,d86,d87,...,c17,c18,c19,c20,c21,c22,luclms,guclms,cez,city
0,1980,166746.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,12.024227,NaN,NaN,1
1,1981,83561.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,11.333332,-0.690895,0.0,1
2,1982,158146.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,11.971274,0.637942,0.0,1
3,1983,83572.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,11.333464,-0.637811,0.0,1
4,1984,45949.0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,10.735288,-0.598176,1.0,1


In [ ]:
# Panel de ciudades como zonas empresariales desde 1980 a 1988
# 12 de las ciudades no recibieron la designación de zona empresarial en el período y sirven como grupo de control
# Se relaciona la designación como zona empresarial con los reclamos de seguro de desempleo (uclms)

In [ ]:
# El modelo básico para la evalución de la politica es 

$log(uclms_{it}) = \theta_{t} + \beta_{1} ez_{it} + a_{i} + u_{it}$

In [ ]:
# La ordenada es difrente para cada período y busca reflejar que los reclamos descienden en todas las ciudades en el período
# ezit es una dummy de en ese período la ciudad era definida como zona empresarial
# ai son los efectos fijos por ciudad
# ai y ezi probablemente se correlacionan positivamente pues se define ez cuando hay problemas económicos
# Por esto se debe diferenciar para eliminar ai

$ \Delta log(uclms_{it}) = \alpha_{0} + \alpha_{1} d82_{t} +...+\alpha_{7} d88_{t} + \beta_{1} \Delta ez_{it} + \Delta u_{it}$

In [ ]:
# La dependeinte es aproximadamente la tasa de crecimiento de los reclamos de desempleo de t-1 a t 
# La estimación se hace para los años 81 a 88

In [101]:
regdfej13_8 = smf.ols('guclms ~  cez + d82+ d83+ d84+ d85 + d86 +d87+d88  ', data = dfej13_8).fit()
print(regdfej13_8.summary())

                            OLS Regression Results                            
Dep. Variable:                 guclms   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     34.50
Date:                Mon, 12 Oct 2020   Prob (F-statistic):           1.08e-31
Time:                        10:36:18   Log-Likelihood:                 24.553
No. Observations:                 176   AIC:                            -31.11
Df Residuals:                     167   BIC:                            -2.573
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3216      0.046     -6.982      0.0

In [ ]:
# Comentario : La presencia de la designación de ez provoca que se reduzcan los reclamos 
# Como está en log la dependiente y es una binaria ez , el valor en unidades nominales es con exponeneciación

In [104]:
np.exp(regdfej13_8.params[1])-1

-0.16629657463852832

In [ ]:
# La reducción en los reclamos se reducen 16% por la designación como ez y es significativo el efecto

In [ ]:
# Test de heterocedasticidad 

In [105]:
bptest = sms.diagnostic.het_breuschpagan(regdfej13_8.resid, regdfej13_8.model.exog)
bptest2 = pd.DataFrame({'Chi-Sq':[bptest[0]],
                   'Prob>Chi-Sq':[bptest[1]]})
print(bptest2)

     Chi-Sq  Prob>Chi-Sq
0  6.913966     0.545943


In [ ]:
# No hay evidencia de heterocedasticidad

In [106]:
#Falta lo de autocorrelación

## Ejemplo 13.9 Tasas de delincuencia por condado en Carolina del Norte (crime4.dta)

In [115]:
dfej13_9 = pd.read_stata('crime4.dta')
dfej13_9.head(10)

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctymle,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,-2.433870,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,-2.449038,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,-2.464036,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,-2.478925,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,-2.497306,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125
5,1,86,0.034752,0.326062,0.512324,0.439863,6.90,0.001895,2.385681,29.740980,...,-2.524721,3.006608,-0.051062,0.002048,-0.121867,-0.122245,0.052056,-0.015258,0.055322,0.042765
6,1,87,0.035604,0.298270,0.527596,0.436170,6.71,0.001828,2.422633,30.993681,...,-2.552702,3.006608,0.024198,-0.089089,0.029374,-0.008431,-0.027923,-0.036189,0.041257,-0.193899
7,3,81,0.016392,0.202899,0.869048,0.465753,8.45,0.000594,0.976834,14.560878,...,-2.441794,2.068926,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,82,0.019065,0.162218,0.772152,0.377049,5.71,0.000705,0.992278,35.640728,...,-2.447933,2.068926,0.151060,-0.223767,-0.118217,-0.211280,-0.391948,0.170985,0.895151,-0.170775
9,3,83,0.015149,0.181586,1.028170,0.438356,8.69,0.000659,1.003861,19.261877,...,-2.454076,2.068926,-0.229912,0.112788,0.286354,0.150656,0.419954,-0.067522,-0.615361,0.231241


In [ ]:
# Panel de 90 condados de Carolina del Norte, de los años 1981 al 87
# Se estima un modelo para explicar las tasas de crimen en log con
# prbarr = probabilidad de arresto
# prbconv = probabilidad de condena
# prbpris = probabilidad de prisión
# avgsen = promedio sentencia
# polpc = policías per cápita
# Se utilizan log para estimar elasticidades
# Se incluyen las dummies temporales para controlar las tendencias de todos los condados

$ \Delta log(crmrte_{it}) = \alpha_{0} +\alpha_{1} d83_{t} +...+\alpha_{7} d87_{t} +\beta_{1} \Delta log(prbarr)_{it}
+ \beta_{2} \Delta log(prbconv)_{it}
+ \beta_{3} \Delta log(prbpris)_{it}
+ \beta_{4} \Delta log(avgsen)_{it}
+ \beta_{5} \Delta log(polpc)_{it}
+ \Delta u_{it}$

In [ ]:
# El panel ya está construido con los rezagos de las variables 

In [117]:
dfej13_9[['county','year','prbarr','lprbarr','clprbarr']].head(10)

,county,year,prbarr,lprbarr,clprbarr
0,1,81,0.289696,-1.238923,NaN
1,1,82,0.338111,-1.084381,0.154542
2,1,83,0.330449,-1.107303,-0.022922
3,1,84,0.362525,-1.014662,0.092641
4,1,85,0.325395,-1.122715,-0.108054
5,1,86,0.326062,-1.120668,0.002048
6,1,87,0.298270,-1.209756,-0.089089
7,3,81,0.202899,-1.595047,NaN
8,3,82,0.162218,-1.818814,-0.223767
9,3,83,0.181586,-1.706026,0.112788


In [118]:
-1.084381+1.238923

0.15454199999999996

In [ ]:
# Obviamente no puede hacerse simplemente haciendo el lag en la columna lprbarr, sino que se hace por grupos de county

In [120]:
dfej13_9.groupby('county')['lprbarr'].diff()[0:10]

0         NaN
1    0.154542
2   -0.022922
3    0.092641
4   -0.108054
5    0.002048
6   -0.089089
7         NaN
8   -0.223767
9    0.112788
Name: lprbarr, dtype: float32

In [94]:
dfej13_9.dtypes

county        int16
year           int8
crmrte      float32
prbarr      float32
prbconv     float32
prbpris     float32
avgsen      float32
polpc       float32
density     float32
taxpc       float32
west           int8
central        int8
urban          int8
pctmin80    float32
wcon        float32
wtuc        float32
wtrd        float32
wfir        float32
wser        float32
wmfg        float32
wfed        float32
wsta        float32
wloc        float32
mix         float32
pctymle     float32
d82            int8
d83            int8
d84            int8
d85            int8
d86            int8
d87            int8
lcrmrte     float32
lprbarr     float32
lprbconv    float32
lprbpris    float32
lavgsen     float32
lpolpc      float32
ldensity    float32
ltaxpc      float32
lwcon       float32
lwtuc       float32
lwtrd       float32
lwfir       float32
lwser       float32
lwmfg       float32
lwfed       float32
lwsta       float32
lwloc       float32
lmix        float32
lpctymle    float32


In [103]:
regdfej13_9 = smf.ols('clcrmrte ~   d83+ d84+ d85 + d86 +d87+ clprbarr + clprbcon + clprbpri + clavgsen + clpolpc  ', data = dfej13_9).fit()
print(regdfej13_9.summary())

                            OLS Regression Results                            
Dep. Variable:               clcrmrte   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     40.32
Date:                Mon, 12 Oct 2020   Prob (F-statistic):           6.30e-59
Time:                        10:40:08   Log-Likelihood:                 248.48
No. Observations:                 540   AIC:                            -475.0
Df Residuals:                     529   BIC:                            -427.7
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0077      0.017      0.452      0.6

In [121]:
regdfej13_9a = smf.ols('clcrmrte ~   d83+ d84+ d85 + d86 +d87+ clprbarr + clprbcon + clprbpri + clavgsen + clpolpc  ', data = dfej13_9).fit(cov_type='HC1')
print(regdfej13_9a.summary())

                            OLS Regression Results                            
Dep. Variable:               clcrmrte   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     15.83
Date:                Mon, 12 Oct 2020   Prob (F-statistic):           5.38e-25
Time:                        14:40:14   Log-Likelihood:                 248.48
No. Observations:                 540   AIC:                            -475.0
Df Residuals:                     529   BIC:                            -427.7
Df Model:                          10                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0077      0.015      0.527      0.5

In [122]:
print(summary_col([regdfej13_9, regdfej13_9a],stars=True,float_format='%0.3f',
                  model_names=['Hetrosced\n(b/se)', 'Robust\n(b/se)'],
                 info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.3f}".format(x.rsquared), 
                           'Adj.R2':lambda x: "{:.3f}".format(x.rsquared_adj)}))


          Hetrosced   Robust 
            (b/se)    (b/se) 
-----------------------------
Intercept 0.008     0.008    
          (0.017)   (0.015)  
d83       -0.100*** -0.100***
          (0.024)   (0.022)  
d84       -0.048**  -0.048** 
          (0.024)   (0.020)  
d85       -0.005    -0.005   
          (0.023)   (0.024)  
d86       0.028     0.028    
          (0.024)   (0.021)  
d87       0.041*    0.041*   
          (0.024)   (0.024)  
clprbarr  -0.327*** -0.327***
          (0.030)   (0.051)  
clprbcon  -0.238*** -0.238***
          (0.018)   (0.031)  
clprbpri  -0.165*** -0.165***
          (0.026)   (0.035)  
clavgsen  -0.022    -0.022   
          (0.022)   (0.025)  
clpolpc   0.398***  0.398*** 
          (0.027)   (0.076)  
N         540       540      
R2        0.433     0.433    
Adj.R2    0.422     0.422    
Standard errors in
parentheses.
* p<.1, ** p<.05, ***p<.01


In [ ]:
# Comentario : La estiación de la ecuación en diferencias muestra eu los arrestos, condenas y tiempo en prisión  tienen 
# signos esperados y son significativas
# 1% de aumento en probabilidad de arresto  disminuye la delincuencia en 0.33%
# La sentencia promedio muestra poco efecto y no significativo
# Es notable que el aumento del 1% de policías aumenta la tasa de delincuencia en 0.4% , y es significativo.
# El problema puede ser que la tasa de delincuencia se toma de delitos reportados y que cuando hay más policías se reportan más
# O bien la endogeneidad de la cantidad de policías puede ser que aumenten cuando esperan más crímenes, por lo tanto no puede
# interpretarse la ecuación causalmente (ver Varaibles instrumentales)

In [ ]:
# Heterocedasticiad y autocorrelación

In [123]:
bptest = sms.diagnostic.het_breuschpagan(regdfej13_9.resid, regdfej13_9.model.exog)
bptest2 = pd.DataFrame({'Chi-Sq':[bptest[0]],
                   'Prob>Chi-Sq':[bptest[1]]})
print(bptest2)

      Chi-Sq  Prob>Chi-Sq
0  10.929708     0.363021


In [ ]:
# FALTA White y autocorrelación

In [126]:
from statsmodels.compat import lzip

In [127]:
W_t = sms.diagnostic.het_white(regdfej13_9.resid, regdfej13_9.model.exog)
print(pd.DataFrame(lzip(['LM statistic', 'p-value',  'F-value', 'F: p-value'], W_t)))

              0             1
0  LM statistic  2.566499e+02
1       p-value  1.461931e-29
2       F-value  8.858428e+00
3    F: p-value  7.256697e-43
